# Extraction

Outra aplicação comum para funções é a relização de extração de conteúdos do texto informado. Isso facilita o parseamento de informações em grande escala, em que podemos utilizar scripts para limpar textos informados, mantendo apenas o necessário da informação que o usuário necessita. Vamos começar com um exemplo simples, extraindo a informação de datas e acontecimentos de um texto.

## Extraindo datas e acontecimentos

Digamos que temos um texto como o a seguir e queremos extrair datas e acontecimentos que aparecerem no texto:


In [1]:
texto = '''A Apple foi fundada em 1 de abril de 1976 por Steve Wozniak, Steve Jobs e Ronald Wayne 
com o nome de Apple Computers, na Califórnia. O nome foi escolhido por Jobs após a visita do pomar 
de maçãs da fazenda de Robert Friedland, também pelo fato do nome soar bem e ficar antes da Atari 
nas listas telefônicas.

O primeiro protótipo da empresa foi o Apple I que foi demonstrado na Homebrew Computer Club em 1975, 
as vendas começaram em julho de 1976 com o preço de US$ 666,66, aproximadamente 200 unidades foram 
vendidas,[21] em 1977 a empresa conseguiu o aporte de Mike Markkula e um empréstimo do Bank of America.'''

In [2]:
from pydantic import BaseModel, Field #Importação atualizada
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function

class Acontecimento(BaseModel):
    '''Informação sobre um acontecimento'''
    data: str = Field(description='Data do acontecimento no formato YYYY-MM-DD')
    acontecimento: str = Field(description='Acontecimento ecxtraído do texto')

class ListaAcontecimentos(BaseModel):
    """Acontecimentos para extração"""
    acontecimentos: List[Acontecimento] = Field(description='Lista de acontecimentos presentes no texto informado')

tool_acontecimentos = convert_to_openai_function(ListaAcontecimentos)
tool_acontecimentos

{'name': 'ListaAcontecimentos',
 'description': 'Acontecimentos para extração',
 'parameters': {'properties': {'acontecimentos': {'description': 'Lista de acontecimentos presentes no texto informado',
    'items': {'description': 'Informação sobre um acontecimento',
     'properties': {'data': {'description': 'Data do acontecimento no formato YYYY-MM-DD',
       'type': 'string'},
      'acontecimento': {'description': 'Acontecimento ecxtraído do texto',
       'type': 'string'}},
     'required': ['data', 'acontecimento'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['acontecimentos'],
  'type': 'object'}}

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extraia as frases de acontecimentos. Elas devem ser extraídas integralmente'),
    ('user', '{input}')
])
chat = ChatOpenAI()

chain = (prompt 
         | chat.bind(functions=[tool_acontecimentos], function_call={'name': 'ListaAcontecimentos'}))


In [4]:
chain.invoke({'input': texto})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"acontecimentos":[{"data":"1976-04-01","acontecimento":"A Apple foi fundada por Steve Wozniak, Steve Jobs e Ronald Wayne na Califórnia com o nome de Apple Computers."},{"data":"1975","acontecimento":"Demonstração do primeiro protótipo da empresa, o Apple I, na Homebrew Computer Club."},{"data":"1976-07","acontecimento":"Início das vendas do Apple I com o preço de US$ 666,66."},{"data":"1977","acontecimento":"A empresa recebeu o aporte de Mike Markkula e um empréstimo do Bank of America."}]}', 'name': 'ListaAcontecimentos'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 325, 'total_tokens': 469, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'fin

In [5]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (prompt 
         | chat.bind(functions=[tool_acontecimentos], function_call={'name': 'ListaAcontecimentos'})
         | JsonOutputFunctionsParser())

chain.invoke({'input': texto})

{'acontecimentos': [{'data': '1976-04-01',
   'acontecimento': 'A Apple foi fundada por Steve Wozniak, Steve Jobs e Ronald Wayne, com o nome de Apple Computers, na Califórnia.'},
  {'data': '1975-00-00',
   'acontecimento': 'O primeiro protótipo da empresa, o Apple I, foi demonstrado na Homebrew Computer Club.'},
  {'data': '1976-07-00',
   'acontecimento': 'As vendas do Apple I começaram em julho de 1976 com o preço de US$ 666,66.'},
  {'data': '1977-00-00',
   'acontecimento': 'A empresa conseguiu o aporte de Mike Markkula e um empréstimo do Bank of America.'}]}

In [6]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (prompt 
         | chat.bind(functions=[tool_acontecimentos], function_call={'name': 'ListaAcontecimentos'})
         | JsonKeyOutputFunctionsParser(key_name='acontecimentos'))

chain.invoke({'input': texto})

[{'data': '1976-04-01',
  'acontecimento': 'A Apple foi fundada por Steve Wozniak, Steve Jobs e Ronald Wayne com o nome de Apple Computers na Califórnia.'},
 {'data': '1975-07',
  'acontecimento': 'As vendas do Apple I começaram em julho de 1976 com o preço de US$ 666,66 e aproximadamente 200 unidades foram vendidas.'},
 {'data': '1977',
  'acontecimento': 'A empresa conseguiu o aporte de Mike Markkula e um empréstimo do Bank of America.'}]

## Extraindo informações da web

A aplicação de extração pode ser muito utilizanda quando combinada com técnicas de WebScraping. Em geral, em WebScraping estamos buscando informações em páginas web. Em sua grande maioria, essas informações virão comletamente desformatadas e em html, o que dificulta a utilização da informação. Podemos criar aplicações utilizando as técnicas que aprendemos para conseguir as informações específicas que precisamos das páginas que estamos analisando.

Vamos dar um exemplo analisando a página de blog da Asimov e tentando extrair todas os posts contidos na página.
https://hub.asimov.academy/blog/


In [7]:
from langchain_community.document_loaders.web_base import WebBaseLoader

loader = WebBaseLoader('https://hub.asimov.academy/blog/')
page = loader.load()
page

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://hub.asimov.academy/blog/', 'title': 'Blog de Python, IA e Data Science | Asimov Academy', 'description': 'Confira os principais assuntos de programação em Python, Inteligência Artificial e Data Science no blog da Asimov Academy!', 'language': 'pt-BR'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\nBlog de Python, IA e Data Science | Asimov Academy\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTamanho de fonte\n\nA\nA\nA\nA\n\n\n\nAlto contraste\n\nLigado\nDesligado\n\n\n\nAltura de linha\n\n1\n1.3\n1.5\n1.7\n2.0\n\n\n\n \n\n\n\nPesquisar na plataforma\n\n\n\n\n\nEntrar\nCadastrar\n\n\n\n\n\nBlog\n\n\nArtigos\n\n\nTutoriais\n\n\nMateriais Extras\n\n\nEscreva para nós!\n200xp \n\n\n\n\nBlog da Asimov\n\n        Fique por dentro dos principais assuntos de Python e Inteligência Artificial      \n\n\n\n\n\n133 posts\n\n\n\nPesquisa

In [8]:
from pydantic import BaseModel, Field #Importação atualizada
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function

class BlogPost(BaseModel):
    '''Informações sobre um post de blog'''
    titulo: str = Field(description='O título do post de blog')
    autor: str = Field(description='O autor do post de blog')
    data: str = Field(description='A data de publicação do post de blog')

class BlogSite(BaseModel):
    '''Lista de blog posts de um site'''
    posts: List[BlogPost] = Field(description='Lista de posts de blog do site')

tool_blog = convert_to_openai_function(BlogSite)
tool_blog

{'name': 'BlogSite',
 'description': 'Lista de blog posts de um site',
 'parameters': {'properties': {'posts': {'description': 'Lista de posts de blog do site',
    'items': {'description': 'Informações sobre um post de blog',
     'properties': {'titulo': {'description': 'O título do post de blog',
       'type': 'string'},
      'autor': {'description': 'O autor do post de blog', 'type': 'string'},
      'data': {'description': 'A data de publicação do post de blog',
       'type': 'string'}},
     'required': ['titulo', 'autor', 'data'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['posts'],
  'type': 'object'}}

In [9]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extraia da página todos os posts de blog com autor e data de publicação'),
    ('user', '{input}')
])
chat = ChatOpenAI()
chain = (prompt 
         | chat.bind(functions=[tool_blog], function_call={'name': 'BlogSite'})
         | JsonKeyOutputFunctionsParser(key_name='posts'))


In [10]:
chain.invoke({'input': page})

[{'titulo': 'AutoGen: a era dos agentes aut\\u00f4nomos com IA generativa',
  'autor': 'Rebeca Hon\\u00f3rio',
  'data': '2 dias atr\\u00e1s'},
 {'titulo': 'Como instalar o Anaconda no macOS: tutorial passo a passo',
  'autor': 'Mykael Lima',
  'data': '6 dias atr\\u00e1s'},
 {'titulo': 'Sistemas multiagentes: o que s\\u00e3o, como funcionam e como criar em Python',
  'autor': 'Rebeca Hon\\u00f3rio',
  'data': '6 dias atr\\u00e1s'},
 {'titulo': 'CrewAI: para iniciantes em automa\\u00e7\\u00e3o com Python',
  'autor': 'Rebeca Hon\\u00f3rio',
  'data': '9 dias atr\\u00e1s'},
 {'titulo': 'Como instalar VS Code no Linux: guia completo',
  'autor': 'Heitor Tasso',
  'data': '11 horas atr\\u00e1s'},
 {'titulo': 'Freelancer em programa\\u00e7\\u00e3o: guia para iniciantes',
  'autor': 'Rebeca Hon\\u00f3rio',
  'data': '13 dias atr\\u00e1s'},
 {'titulo': 'Como instalar Anaconda no Linux: tutorial passo a passo',
  'autor': 'Mykael Lima',
  'data': '20 dias atr\\u00e1s'},
 {'titulo': 'O que s\\